In [ ]:
import os
from typing import Any, Literal, TypedDict

from flashrank import Ranker
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.retrievers.document_compressors import (
    DocumentCompressorPipeline,
    EmbeddingsFilter,
    LLMChainExtractor,
)
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_compressors import FlashrankRerank
from langchain_community.document_loaders import TextLoader
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_semantic_cache import ensure_semantic_cache
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph
from pydantic import BaseModel, Field, ValidationError

from utils import display_markdown


class LearningModule(TypedDict):
    query: str


class IntentClassifier(BaseModel):
    intent: Literal["Learning", "Progress Check", "Help Request", "Goal Setting"] = (
        Field(
            description="The user's primary intent, chosen from one of the following: 'Learning', 'Progress Check', 'Help Request', or 'Goal Setting'."
        )
    )
    confidence: float = Field(
        description="A confidence score (between 0.0 and 1.0) indicating how certain the model is about the classified intent."
    )

In [ ]:
# cache = SemanticCache(
#     similarity_threshold=0.5, max_cache_size=1000, memory_cache_size=100
# )

In [ ]:
# cache.clear_cache()
ensure_semantic_cache()

In [ ]:
llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    temperature=0.7,
)

llm_intent = llm.with_structured_output(IntentClassifier)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    """# Intent Classification Task

Classify the user query into exactly ONE category:
- **Learning**: Seeking knowledge, tutorials, explanations
- **Progress Check**: Reviewing status, tracking advancement
- **Help Request**: Troubleshooting, problem-solving assistance
- **Goal Setting**: Planning objectives, defining targets

## Output Format:
Category: [CATEGORY]
Confidence: [0.0-1.0]

## Examples:
Query: "How do I learn Python?"
Category: Learning
Confidence: 0.95

Query: "Am I on track with my fitness goals?"
Category: Progress Check
Confidence: 0.90

## User Query:
{query}"""
)

chain = prompt | llm_intent

In [ ]:
# chain.invoke("How can i be perfect for agentic ai interviews in the next 3 months?")  # type: ignore

# The Learning query preprocessing
## Knowledge base creation

In [ ]:
def document_processor(file_path: str) -> list[Document]:
    if not os.path.exists(file_path):
        print("The file path provided does not exist. Please check it again and retry")
        return None

    loader = TextLoader(file_path)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(docs)

    return docs

In [ ]:
docs = document_processor("romeo.txt")

In [ ]:
def create_store(docs: list[Document], persist_dir: str, embeddings):
    if os.path.exists(persist_dir):
        print(f"Loaded the vector store from {persist_dir}")
        store = FAISS.load_local(
            persist_dir, embeddings, allow_dangerous_deserialization=True
        )

    else:
        if not docs:
            print("No documents were passed")
            return None

        if not os.path.exists(persist_dir):
            os.makedirs(persist_dir)
            print(f"Successfuly created the persist directory at {persist_dir}")

        print(f"Creating the new vector store with {len(docs)} documents")
        store = None

        for i in range(0, len(docs), 50):
            batch = docs[i : i + 50]

            print(
                f"Processing batch {i//50 + 1}/{(len(docs) + 49)//50} ({len(batch)} documents)"
            )
            if store is None:
                store = FAISS.from_documents(batch, embeddings)
            else:
                store.add_documents(batch)

        store.save_local(persist_dir)
        print(f"Vector store created and saved at {persist_dir}")

    return store

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-base", model_kwargs={"local_files_only": True}
)

In [ ]:
store = create_store(docs, "faiss_store", embeddings)

In [ ]:
ranker = Ranker(
    model_name="ms-marco-MiniLM-L-12-v2",
    cache_dir=os.path.expanduser("~/.cache/flashrank"),
)

In [ ]:
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 10
retriever = store.as_retriever(search_kwargs={"k": 10})  # type: ignore
retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [ ]:
retriever = store.as_retriever(search_kwargs={"k": 10})  # type: ignore
retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [ ]:
compressor_pipeline = DocumentCompressorPipeline(
    transformers=[
        EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.6),
        EmbeddingsRedundantFilter(embeddings=embeddings, similarity_threshold=0.95),
        FlashrankRerank(client=ranker),
        LLMChainExtractor.from_llm(llm),
    ]
)

In [ ]:
# compressor = FlashrankRerank(client=ranker)
retriever = ContextualCompressionRetriever(
    base_compressor=compressor_pipeline, base_retriever=retriever
)

In [ ]:
class LearningState(TypedDict):
    query: str
    qa_result: str
    relevant_docs: list[Document]
    learning_response: str
    user_profile: dict[str, Any]
    final_explanation: str
    learning_level: str
    learning_style: str
    difficulty_adjustments: str
    personalized_content: str
    learning_activities: list[str]
    related_topics: list[str]
    fallback_used: bool
    search_method: Literal["knowledge_base", "web_search"]

In [ ]:
tavily_search = TavilySearch(
    max_results=1,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False,
)

In [ ]:
class MockDoc:
    def __init__(self, content, url):
        self.page_content = content
        self.metadata = {"source": url}

In [ ]:
def knowledge_base_search(state: LearningState):
    try:
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True,
        )
        answer = qa_chain.invoke(state["query"])

        # LLM quality check
        if _is_adequate(answer["result"], state["query"]):
            return {
                **state,
                "qa_result": answer["result"],
                "relevant_docs": answer["source_documents"],
                "context_chunks": [
                    doc.page_content for doc in answer["source_documents"]
                ],
                "search_method": "knowledge_base",
                "fallback_used": False,
            }

    except Exception as e:
        print(f"Knowledge base search failed: {str(e)}")

    # Web search fallback
    try:
        web_results = tavily_search.invoke(
            f"Romeo and Juliet by Shakespeare: {state['query']}"
        )
        mock_docs = [
            MockDoc(res["content"], res["url"]) for res in web_results["results"]
        ]
        return {
            **state,
            "qa_result": web_results["answer"],
            "relevant_docs": mock_docs,
            "context_chunks": [res["content"] for res in web_results["results"]],
            "search_method": "web_search",
            "fallback_used": True,
        }
    except Exception as e:
        print(f"Web search failed: {str(e)}")

    return {
        **state,
        "qa_result": "I couldn't find relevant information about this topic.",
        "relevant_docs": [],
        "context_chunks": [],
        "search_method": "none",
        "fallback_used": True,
    }


def _is_adequate(answer: str, query: str) -> bool:
    """LLM judge for answer quality."""
    prompt = f"""Question: {query}
    Answer: {answer}

    Is this answer adequate for the Romeo and Juliet question? Consider if it's specific, relevant, and informative.
    Respond only: YES or NO"""

    try:
        # structured_llm = llm.with_structured_output
        response = llm.invoke(prompt)
        return "YES" in response.content.upper()
    except Exception as e:
        print(f"An Error occurred {str(e)}")
        return len(answer) > 50 and "don't know" not in answer.lower()


def generate_learning_response(state: LearningState):
    if not state.get("relevant_docs"):
        return {
            **state,
            "learning_response": "I couldn't find specific information about this topic in Romeo and Juliet. Please try rephrasing your question or ask about a different aspect of the play.",
        }

    context_text = "\n".join(
        [
            f"Context {i+1}:\n{doc.page_content}"
            for i, doc in enumerate(state.get("relevant_docs", []))
        ]
    )

    prompt_template = """You are a literature tutor for Romeo and Juliet.

    Initial Answer: {qa_result}
    Context: {context}
    User Query: {query}

    Provide an enhanced educational response with quotes, themes, and analysis.

    Enhanced Learning Response:"""

    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm | StrOutputParser()

    learning_response = chain.invoke(
        {
            "qa_result": state["qa_result"],
            "context": context_text,
            "query": state["query"],
        }
    )

    return {**state, "learning_response": learning_response}

In [ ]:
class PersonalizedEngine(BaseModel):
    personalized_content: str = Field(
        description="Adjusted complexity to match the learner's level with a maximum of 300 words."
    )
    learning_activities: list[str] = Field(
        description="A list of at most 5 tailored tasks or exercises to reinforce understanding."
    )
    related_topics: list[str] = Field(
        description="At most 5 topics closely connected to the main content for deeper exploration."
    )
    difficulty_adjustments: str = Field(
        description="Explanation of how the content difficulty was modified."
    )


class UserProfile(BaseModel):
    learning_level: str = "intermediate"
    learning_style: str = "visual"
    interests: list[str] = ["character_analysis", "themes"]
    previous_topics: list[str] = []


def content_personalization_engine(state: dict):
    # Error handling for user profile data
    profile_data = state.get("user_profile", {})
    try:
        profile = UserProfile(**profile_data)
    except ValidationError as e:
        print(f"Invalid profile data: {e}")
        profile = UserProfile()  # Use defaults

    if "learning_response" not in state:
        raise ValueError("learning_response is required in state")
    if "query" not in state:
        raise ValueError("query is required in state")

    learning_response = state["learning_response"]
    if len(learning_response) > 2000:
        learning_response = learning_response[:2000] + "..."

    personalization_prompt = ChatPromptTemplate.from_template(
        """Personalize this Romeo and Juliet content for a {learning_level} learner who prefers {learning_style} learning.

        Original content: {learning_response}

        KEEP ALL RESPONSES SHORT:
        - personalized_content: Rewrite for {learning_level} level (max 300 words)
        - learning_activities: List at most 5 activities (one sentence each)
        - related_topics: List at most 5 topic names only
        - difficulty_adjustments: What you changed (max 50 words)
        """
    )

    llm_structured = llm.with_structured_output(PersonalizedEngine)
    chain = personalization_prompt | llm_structured

    personalization_content = chain.invoke(
        {
            "learning_level": profile.learning_level,
            "learning_style": profile.learning_style,
            "interests": ", ".join(profile.interests),
            "previous_topics": ", ".join(profile.previous_topics),
            "learning_response": state["learning_response"],
            "query": state["query"],
        }
    )

    # Extract structured fields from the response
    return {
        **state,
        "personalized_content": personalization_content.personalized_content,  # type: ignore
        "learning_activities": personalization_content.learning_activities,  # type: ignore
        "related_topics": personalization_content.related_topics,  # type: ignore
        "difficulty_adjustments": personalization_content.difficulty_adjustments,  # type: ignore
        "learning_level": profile.learning_level,
        "learning_style": profile.learning_style,
    }

In [ ]:
def generate_explanation(state: LearningState):
    explanation_prompt = ChatPromptTemplate.from_template(
        """Answer this Romeo and Juliet question for a {learning_level} student:

        Question: {query}

        Content: {personalized_content}

        Activities: {learning_activities}

        Related topics: {related_topics}

        Keep your response under 300 words and make it engaging for a {learning_level} learner.
        """
    )

    chain = explanation_prompt | llm
    final_explanation = chain.invoke(
        {
            "query": state["query"],
            "learning_level": state.get("learning_level", "intermediate"),
            "learning_style": state.get("learning_style", "visual"),
            "personalized_content": state.get("personalized_content", ""),
            "learning_activities": "\n".join(state.get("learning_activities", [])),
            "related_topics": ", ".join(state.get("related_topics", [])),
            "difficulty_adjustments": state.get("difficulty_adjustments", ""),
        }
    )

    return {
        **state,
        "final_explanation": (
            final_explanation.content
            if hasattr(final_explanation, "content")
            else str(final_explanation)
        ),
    }

In [ ]:
# conn = sqlite3.connect("memory.db", check_same_thread=False)
# memory = SqliteSaver(conn)
# config = {"configurable": {"thread_id": "1"}}


workflow = StateGraph(LearningState)

# Add nodes
workflow.add_node("knowledge_base_search", knowledge_base_search)
workflow.add_node("generate_learning_response", generate_learning_response)
workflow.add_node("content_personalization_engine", content_personalization_engine)
workflow.add_node("generate_explanation", generate_explanation)

# Define edges

workflow.add_edge("knowledge_base_search", "generate_learning_response")
workflow.add_edge("generate_learning_response", "content_personalization_engine")
workflow.add_edge("content_personalization_engine", "generate_explanation")
workflow.set_entry_point("knowledge_base_search")

app = workflow.compile()  # Remove memory checkpointing

In [ ]:
result = app.invoke(
    {"query": "What are the key events that lead to Romeo and Juliet’s deaths?"},
    # config=config,
)

2025-07-21 16:58:07,342 - INFO - SQLite cache hit
2025-07-21 16:58:07,348 - INFO - SQLite cache hit
2025-07-21 16:58:07,354 - INFO - SQLite cache hit
2025-07-21 16:58:07,359 - INFO - SQLite cache hit
2025-07-21 16:58:07,365 - INFO - SQLite cache hit
2025-07-21 16:58:07,374 - INFO - SQLite cache hit
2025-07-21 16:58:07,383 - INFO - SQLite cache hit
2025-07-21 16:58:07,391 - INFO - SQLite cache hit


In [ ]:
%load_ext autoreload
%autoreload 2
display_markdown(result, title="Romeo and Juliet Analysis Results")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 🎭 Romeo and Juliet Analysis Results

🎭 **Source:** Romeo & Juliet Knowledge Base

---

## ❓ Question

> **❝ What are the key events that lead to Romeo and Juliet’s deaths? ❞**

## 💡 Qa Result

The key events that lead to Romeo and Juliet's deaths include:

1. **Feud between the Montagues and Capulets**: The longstanding rivalry between the two families sets the stage for conflict and tragedy.

2. **Romeo and Juliet's Secret Marriage**: Despite the feud, Romeo and Juliet fall in love and secretly marry, hoping to unite their families.

3. **Tybalt's Death**: After Tybalt kills Mercutio, Romeo avenges his friend's death by killing Tybalt, which leads to Romeo's banishment from Verona.

4. **Juliet's Despair**: Juliet is devastated by Tybalt's death and Romeo's banishment. In an attempt to avoid marrying Paris, she seeks help from Friar Laurence.

5. **The Plan**: Friar Laurence gives Juliet a potion that will make her appear dead for 42 hours, allowing her to escape her marriage to Paris and reunite with Romeo.

6. **Miscommunication**: The message about Juliet's fake death fails to reach Romeo, leading him to believe she is truly dead.

7. **Romeo's Suicide**: In despair, Romeo buys poison and goes to Juliet's tomb, where he encounters Paris and kills him before taking the poison himself.

8. **Juliet's Awakening**: Juliet awakens shortly after Romeo's death, finds him dead beside her, and takes her own life with his dagger.

These events culminate in the tragic deaths of both Romeo and Juliet,
highlighting the consequences of their families' feud and the miscommunications
that arise from it.

## 📖 Learning Response

Certainly! The tragic deaths of Romeo and Juliet are the culmination of a series
of critical events that intertwine themes of love, fate, and the consequences of
familial conflict. Here’s a detailed analysis of the key events leading to their
untimely demise, supported by relevant quotes and thematic exploration.

### **Key Events Leading to Their Deaths:**

1. **The Feud between the Montagues and Capulets:**
The longstanding rivalry sets the stage for tragedy. The opening brawl in Act 1,
Scene 1, introduces the violent conflict between the two families. This feud not
only fuels hatred but also creates an atmosphere of secrecy and urgency for
Romeo and Juliet’s love. The Prince’s proclamation, “If ever you disturb our
streets again, your lives shall pay the forfeit of the peace,” highlights the
gravity of their conflict and foreshadows the tragic consequences.

2. **Romeo and Juliet’s Secret Marriage:**
Romeo and Juliet fall in love at the Capulet's feast and secretly marry the next
day, hoping to reconcile their families. Friar Laurence’s line, “For this
alliance may so happy prove, to turn your households' rancor to pure love” (Act
2, Scene 3), emphasizes their hope that love can overcome hatred. However, this
act of love is shrouded in secrecy and defiance, setting up a tragic irony.

3. **Tybalt’s Death:**
The turning point occurs when Tybalt kills Mercutio, and in a fit of rage, Romeo
avenges his friend’s death by killing Tybalt. This act leads to Romeo’s
banishment. Romeo laments, “O, I am fortune’s fool!” (Act 3, Scene 1),
suggesting that he feels trapped by fate. This quote encapsulates the theme of
fate versus free will and marks the beginning of the tragic chain reaction.

4. **Juliet’s Despair:**
Juliet’s world crumbles as she learns of Tybalt’s death and Romeo’s banishment.
Her despair is palpable when she exclaims, “O serpent heart, hid with a
flowering face!” (Act 3, Scene 2), reflecting her internal conflict between love
and grief. This moment highlights the theme of duality in love—its beauty and
its potential for destruction.

5. **The Plan:**
In a desperate attempt to avoid marrying Paris, Juliet seeks Friar Laurence’s
help. He provides her with a potion that will make her appear dead for 42 hours.
The plan, though well-intentioned, is fraught with risks. It reveals the theme
of miscommunication and the extent to which the characters are driven by their
circumstances.

6. **Miscommunication:**
The failure of the message to reach Romeo is a critical moment that seals their
fate. Instead of receiving news of Juliet's feigned death, Romeo learns of her
supposed demise. This miscommunication illustrates the fragility of human
connections and the devastating impact of societal constraints.

7. **Romeo’s Suicide:**
Believing Juliet is truly dead, Romeo buys poison and goes to her tomb. His
despair is encapsulated in the line, “Here’s to my love!” (Act 5, Scene 3) as he
drinks the poison, showcasing the theme of love as a force that drives
individuals to despair. His actions underscore the tragic consequences of
impulsive decisions in moments of extreme emotion.

8. **Juliet’s Awakening:**
When Juliet awakens and finds Romeo dead, her grief leads her to take her own
life with his dagger. Her final words, “O happy dagger!” (Act 5, Scene 3),
painfully juxtapose love and death, cementing the theme of love's tragic nature.
The imagery of blood and violence in their deaths encapsulates the destructive
impact of the feud.

### **Thematic Analysis:**

- **Love vs. Hate:** The juxtaposition of love and hate is central to the play. The intense love between Romeo and Juliet exists against the backdrop of their families’ hatred, ultimately leading to their destruction.

- **Fate and Free Will:** The characters frequently grapple with their destinies. Romeo’s lamentation about being “fortune’s fool” suggests an awareness of fate’s role in their lives, raising questions about the extent of their control over their fates.

- **Miscommunication and Consequences:** The constant miscommunication throughout the play highlights how crucial information can be lost in times of crisis. The tragic outcomes stem from a series of misunderstandings, emphasizing the importance of communication in human relationships.

### **Conclusion:**
The deaths of Romeo and Juliet serve as a poignant reminder of the destructive
potential of hate and the tragic consequences of familial conflict. Through
their love story, Shakespeare explores profound themes of love, fate, and the
human condition, leaving a lasting impact on audiences and literature alike.
Their tragedy ultimately calls for reflection on the ways in which societal
divisions can obliterate personal happiness.

## 🎯 Final Explanation

In Shakespeare's "Romeo and Juliet," the tragic deaths of the two young lovers
stem from a series of dramatic events shaped by love, fate, and family conflict.
Let’s break down the key moments that lead to their heartbreaking end:

1. **Family Feud:** The Montagues (Romeo’s family) and Capulets (Juliet’s family) are bitter enemies. This hatred makes it dangerous for Romeo and Juliet to love each other. The Prince warns everyone that more fighting will lead to severe punishment.

2. **Secret Marriage:** Romeo and Juliet meet at a party and quickly fall in love. They decide to marry in secret, hoping their union will heal the family feud. However, this secrecy causes more problems.

3. **Tybalt’s Death:** When Tybalt, Juliet’s cousin, kills Mercutio (Romeo’s friend), Romeo seeks revenge and kills Tybalt. As a result, he is banished from Verona, which deepens the tragedy.

4. **Juliet’s Grief:** Juliet is devastated by Tybalt’s death and Romeo’s banishment. Her heart is torn between her love for Romeo and her loyalty to her family.

5. **The Plan:** To avoid marrying Paris, Juliet fakes her death using a potion. She hopes this plan will reunite her with Romeo, but it’s risky and relies on good communication.

6. **Miscommunication:** Unfortunately, Romeo never learns about Juliet’s fake death. Instead, he believes she is truly gone.

7. **Tragic End:** In his despair, Romeo takes poison. When Juliet awakens and finds him dead, she takes her own life. Their deaths highlight how love can lead to tragedy, especially when mixed with misunderstandings and family conflict.

Engage with the story through activities like creating a comic strip or a
timeline of these events!

---

## 📚 Additional Information

### 📋 Difficulty Adjustments

I simplified the language and concepts, focusing on clarity and key events while
maintaining the original themes.

### ✅ Learning Activities

✅ Create a comic strip depicting the key events leading to Romeo and Juliet's deaths.

✅ Draw a visual timeline of the play's main events.

✅ Make a poster highlighting the themes of love, fate, and miscommunication.

✅ Watch a movie adaptation of the play and compare it to the text.

✅ Develop a character map showing relationships and conflicts among characters.

### 📚 Learning Level

intermediate

### 🎨 Learning Style

visual

### 📋 Personalized Content

In the play "Romeo and Juliet," the deaths of the two lovers are caused by a mix
of love, fate, and family conflict. Here are the main events that lead to their
tragic end:

1. **Family Feud:** The Montagues and Capulets hate each other. This conflict creates problems for Romeo and Juliet’s love. The Prince warns them that further violence will have serious consequences.

2. **Secret Marriage:** Romeo and Juliet fall in love and marry secretly, hoping to end the feud. Friar Laurence believes their love can bring peace, but their secret creates tension.

3. **Tybalt’s Death:** When Tybalt kills Mercutio, Romeo is enraged and kills Tybalt, leading to his banishment. He feels trapped by fate, which begins the tragic events.

4. **Juliet’s Grief:** Juliet is heartbroken when she hears about Tybalt’s death and Romeo’s banishment. She feels torn between love and sorrow.

5. **The Plan:** To escape an unwanted marriage to Paris, Juliet takes a potion to fake her death. However, this risky plan leads to miscommunication.

6. **Miscommunication:** Romeo does not receive the message about Juliet’s plan and believes she’s truly dead. This misunderstanding seals their fate.

7. **Tragic End:** In despair, Romeo takes poison, believing Juliet is dead. When Juliet awakens and finds him dead, she takes her life too. Their deaths illustrate the tragic results of love and conflict.

### 🔗 Related Topics

🎭 Shakespeare's Themes

🎭 Family Feuds in Literature

🎭 Tragedy in Drama

🎭 Love and Loss

🎭 The Role of Fate

### 📜 Relevant Docs

### 📜 Text Reference 1
---
📖 **Source:** romeo.txt

> *"FIRST WATCH.
Sovereign, here lies the County Paris slain,
And Romeo dead, and Juliet, dead before,
Warm and new kill’d.

PRINCE.
Search, seek, and know how this foul murder comes.

CAPULET.
O heaven! O wife, look how our daughter bleeds!
This dagger hath mista’en, for lo, his house
Is empty on the b..."*

### 📜 Text Reference 2
---
📖 **Source:** romeo.txt

> *"Romeo, there dead, was husband to that Juliet,  
And she, there dead, that Romeo’s faithful wife.  
I married them; and their stol’n marriage day  
Was Tybalt’s doomsday, whose untimely death  
Banish’d the new-made bridegroom from this city;  
For whom, and not for Tybalt, Juliet pin’d."*

### 📜 Text Reference 3
---
📖 **Source:** romeo.txt

> *"NURSE.  
Tybalt is gone, and Romeo banished,  
Romeo that kill’d him, he is banished.  

JULIET.  
O God! Did Romeo’s hand shed Tybalt’s blood?"*

# Goal setting

In [ ]:
class AgentState(TypedDict):
    query: str
    goal: str
    smart_goal: str
    curriculum: list[str]
    learning_path: list[str]
    requirements: list[str]

In [ ]:
GOAL_SETTING_TEMPLATE = """You are an educational assistant. 
Create a clear, basic learning goal from the user's input.

User Input: {input}"""

SMART_GOAL_TEMPLATE = """Transform this goal into SMART format with specific metrics:
- Specific: Exactly what will be accomplished?
- Measurable: How will you track progress? (e.g., "complete 5 projects", "score 80% on exam")
- Achievable: Realistic timeframe and scope
- Relevant: Why this matters for the user's objectives  
- Time-bound: Specific deadline (e.g., "within 3 months")

Goal: {goal}"""

LEARNING_PATH_TEMPLATE = """Create a sequential learning path to achieving this SMART Goal
                        Break it down into logical, progressive steps that build upon each other.
                        SMART GOAL: {smart_goal}
                       
                        """

CURRICULUM_TEMPLATE = """Design a curriculum structure aligned with this learning path.
Create modules that support the learning objectives and path.

Learning Path: {learning_path}
SMART Goal: {smart_goal}
"""

PREREQUISITES_TEMPLATE = """Identify the prerequisites needed before starting this curriculum.
Consider knowledge, skills, tools, or resources required.

Curriculum: {curriculum}
SMART Goal: {smart_goal}
"""

In [ ]:
class Goal(BaseModel):
    goal: str = Field(description="A structured well formatted goal to be achieved")


class SmartGoal(BaseModel):
    smart_goal: str = Field(description="A SMART formatted goal with specific timeline")


class LearningPath(BaseModel):
    learning_path: list[str] = Field(
        description="Sequential learning steps to achieve the goal"
    )


class Curriculum(BaseModel):
    modules: list[str] = Field(
        description="Curriculum modules aligned with learning objectives"
    )


class Prerequisites(BaseModel):
    requirements: list[str] = Field(
        description="Prerequisites required before starting on the goal"
    )

In [ ]:
def goal_setting_node(state: AgentState):
    prompt = ChatPromptTemplate.from_template(GOAL_SETTING_TEMPLATE)
    structured_llm = llm.with_structured_output(Goal)
    chain = prompt | structured_llm

    result = chain.invoke({"input": state["query"]})
    return {**state, "goal": result.goal}


def smart_goal_node(state: AgentState):
    prompt = ChatPromptTemplate.from_template(SMART_GOAL_TEMPLATE)
    structured_llm = llm.with_structured_output(SmartGoal)

    chain = prompt | structured_llm

    result = chain.invoke({"goal": state["goal"]})
    return {**state, "smart_goal": result.smart_goal}


def learning_path_node(state: AgentState):
    prompt = ChatPromptTemplate.from_template(LEARNING_PATH_TEMPLATE)
    structured_llm = llm.with_structured_output(LearningPath)

    chain = prompt | structured_llm

    result = chain.invoke({"smart_goal": state["smart_goal"]})
    return {**state, "learning_path": result.learning_path}


def curriculum_node(state: AgentState):
    prompt = ChatPromptTemplate.from_template(CURRICULUM_TEMPLATE)
    structured_llm = llm.with_structured_output(Curriculum)

    chain = prompt | structured_llm

    result = chain.invoke(
        {"learning_path": state["learning_path"], "smart_goal": state["smart_goal"]}
    )
    return {**state, "curriculum": result.modules}


def prerequisite_checker_node(state: AgentState):
    prompt = ChatPromptTemplate.from_template(PREREQUISITES_TEMPLATE)
    structured_llm = llm.with_structured_output(Prerequisites)

    chain = prompt | structured_llm

    result = chain.invoke(
        {"curriculum": state["curriculum"], "smart_goal": state["smart_goal"]}
    )
    return {**state, "requirements": result.requirements}

In [ ]:
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("goal_setting_node", goal_setting_node)
workflow.add_node("smart_goal_node", smart_goal_node)
workflow.add_node("learning_path_node", learning_path_node)
workflow.add_node("curriculum_node", curriculum_node)
workflow.add_node("prerequisite_checker_node", prerequisite_checker_node)

# Add edges
workflow.add_edge("goal_setting_node", "smart_goal_node")
workflow.add_edge("smart_goal_node", "learning_path_node")
workflow.add_edge("learning_path_node", "curriculum_node")
workflow.add_edge("curriculum_node", "prerequisite_checker_node")

workflow.set_entry_point("goal_setting_node")
workflow.set_finish_point("prerequisite_checker_node")

graph = workflow.compile()

In [ ]:
graph

In [ ]:
user_query = "I need to learn machine learning in 6 months"
initial_state = {
    "query": user_query,
    "goal": "",
    "smart_goal": "",
    "learning_path": [],
    "curriculum": [],
    "prerequisites": [],
}

In [ ]:
goals = graph.invoke(initial_state)

In [ ]:
%load_ext autoreload
%autoreload 2
display_markdown(goals, title="Goals Analysis")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 🎭 Goals Analysis

## ❓ Question

> **❝ I need to learn machine learning in 6 months ❞**

---

## 📚 Additional Information

### 📋 Curriculum

## 📋 Module 1: Introduction to Machine Learning
  - Overview of Machine Learning
  - Types of Machine Learning: Supervised vs Unsupervised
  - Applications of Machine Learning

## 📋 Module 2: Data Preprocessing and Feature Engineering
  - Understanding Data Types and Structures
  - Data Cleaning Techniques
  - Feature Scaling and Normalization
  - Feature Engineering Best Practices

## 📋 Module 3: Supervised Learning Algorithms
  - Introduction to Linear Regression
  - Exploring Decision Trees and Random Forests
  - Support Vector Machines: Theory and Applications
  - Evaluating Model Performance and Metrics

## 📋 Module 4: Unsupervised Learning Algorithms
  - Clustering Techniques: K-Means and Hierarchical Clustering
  - Dimensionality Reduction: PCA and t-SNE
  - Anomaly Detection and Its Applications

## 📋 Module 5: Neural Networks and Deep Learning
  - Introduction to Neural Networks
  - Understanding Activation Functions
  - Training Neural Networks: Backpropagation and Optimization

## 📋 Module 6: Practical Applications and Project Development
  - Identifying a Project Idea and Dataset
  - Data Gathering and Preprocessing for Projects
  - Step-by-Step Model Development

## 📋 Module 7: Model Evaluation and Improvement
  - Techniques for Model Evaluation
  - Hyperparameter Tuning and Optimization
  - Iterating on Feedback for Model Improvement

## 📋 Module 8: Project Documentation and Presentation
  - Documenting the Project Process
  - Preparing a Comprehensive Report
  - Presentation Skills: Sharing Findings and Insights

### 📋 Goal

To gain a foundational understanding of machine learning concepts, algorithms,
and applications, completing at least one practical project, within a 6-month
timeframe.

### 📋 Learning Path

- Research and choose a reputable online machine learning course (e.g., Coursera, edX, or Udacity) that covers foundational concepts and algorithms.

- Enroll in the selected online course and set a study schedule to complete the course within 6 months, allocating time for both learning and assessments.

- Begin with the introductory modules of the course to understand basic concepts such as supervised and unsupervised learning, data preprocessing, and feature engineering.

- Progress to more advanced modules covering key algorithms like linear regression, decision trees, support vector machines, and neural networks.

- Complete all quizzes and assignments associated with each module, aiming for at least 80% on each assessment to ensure comprehension and retention of the material.

- Join online forums or study groups related to the course to discuss concepts, solve problems collaboratively, and enhance understanding through peer learning.

- After completing the course material, identify a practical project idea that applies the concepts learned (e.g., building a predictive model using a dataset from Kaggle).

- Gather and preprocess the data needed for the project, applying the techniques learned in the course.

- Develop the machine learning model step-by-step, starting with data exploration, feature selection, model training, and evaluation.

- Document the project process, including challenges faced and solutions implemented, to reinforce learning and create a portfolio piece.

- Seek feedback on the project from peers or mentors, and iterate on the model to improve its performance based on their suggestions.

- Finalize the project by preparing a presentation or report that explains the project's objectives, methodologies, results, and conclusions, demonstrating your understanding of machine learning concepts.

### 📋 Requirements

- Basic understanding of programming concepts, preferably in Python or R.

- Familiarity with basic statistics and probability concepts.

- Knowledge of linear algebra and calculus fundamentals.

- Access to a computer with a stable internet connection and the ability to install software.

- Familiarity with data visualization tools (e.g., Matplotlib, Seaborn) is beneficial.

- Basic knowledge of data handling libraries (e.g., Pandas, NumPy) in Python.

### 📋 Smart Goal

By the end of 6 months, I will gain a foundational understanding of machine
learning concepts and algorithms by completing an online course with a minimum
score of 80% on all assessments, and I will apply this knowledge by developing
and completing at least one practical machine learning project that demonstrates
my understanding.